# Python for Biomedical Engineering Applications

This notebook covers essential Python programming concepts and libraries specifically tailored for biomedical engineering applications. We'll explore functions, classes, and libraries commonly used in processing and analyzing biomedical data.

## 1. Functions for Biomedical Data Processing

Functions are essential building blocks for organizing code in biomedical applications. They allow us to:
- Create reusable components for common biomedical calculations
- Encapsulate complex algorithms for signal processing
- Standardize data transformation steps
- Document biomedical formulas and implementations

In [ ]:
# Example: Basic physiological calculations as functions

def calculate_bmi(weight_kg, height_m):
    """
    Calculate Body Mass Index (BMI)
    
    Parameters:
    weight_kg (float): Weight in kilograms
    height_m (float): Height in meters
    
    Returns:
    float: BMI value (kg/m²)
    """
    if height_m <= 0 or weight_kg <= 0:
        raise ValueError("Height and weight must be positive values")
    return weight_kg / (height_m ** 2)

def calculate_bsa_dubois(weight_kg, height_cm):
    """
    Calculate Body Surface Area using the DuBois formula
    
    Parameters:
    weight_kg (float): Weight in kilograms
    height_cm (float): Height in centimeters
    
    Returns:
    float: Body surface area in m²
    """
    return 0.007184 * (weight_kg ** 0.425) * (height_cm ** 0.725)

def calculate_ideal_weight(height_cm, gender):
    """
    Calculate ideal body weight using the Devine formula
    
    Parameters:
    height_cm (float): Height in centimeters
    gender (str): 'M' for male, 'F' for female
    
    Returns:
    float: Ideal weight in kg
    """
    height_inches = height_cm / 2.54
    
    if gender.upper() == 'M':
        return 50 + 2.3 * (height_inches - 60)
    elif gender.upper() == 'F':
        return 45.5 + 2.3 * (height_inches - 60)
    else:
        raise ValueError("Gender must be 'M' or 'F'")

# Testing our functions
patient_weight = 70  # kg
patient_height = 175  # cm
patient_height_m = patient_height / 100  # m
patient_gender = 'M'

print(f"Patient BMI: {calculate_bmi(patient_weight, patient_height_m):.2f} kg/m²")
print(f"Patient BSA: {calculate_bsa_dubois(patient_weight, patient_height):.2f} m²")
print(f"Patient ideal weight: {calculate_ideal_weight(patient_height, patient_gender):.2f} kg")

### Signal Processing Functions

Biomedical engineers frequently work with time-series data from various physiological signals (ECG, EEG, EMG, etc.). Let's implement some common signal processing functions.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def moving_average(signal, window_size):
    """
    Apply moving average filter to smooth a signal
    
    Parameters:
    signal (array-like): Input signal
    window_size (int): Size of the moving window
    
    Returns:
    numpy.ndarray: Filtered signal
    """
    return np.convolve(signal, np.ones(window_size)/window_size, mode='valid')

def find_peaks(signal, threshold, min_distance=1):
    """
    Find peaks in a signal above a threshold
    
    Parameters:
    signal (array-like): Input signal
    threshold (float): Minimum amplitude for peak detection
    min_distance (int): Minimum samples between peaks
    
    Returns:
    list: Indices of detected peaks
    """
    peaks = []
    for i in range(1, len(signal)-1):
        if signal[i] > threshold and signal[i] > signal[i-1] and signal[i] > signal[i+1]:
            if not peaks or i - peaks[-1] >= min_distance:
                peaks.append(i)
    return peaks

def calculate_heart_rate(peak_indices, sampling_rate):
    """
    Calculate heart rate from peak indices
    
    Parameters:
    peak_indices (list): Indices of R-peaks in ECG
    sampling_rate (float): Signal sampling rate in Hz
    
    Returns:
    float: Average heart rate in BPM
    """
    if len(peak_indices) < 2:
        return 0
        
    # Calculate intervals between peaks in seconds
    intervals = np.diff(peak_indices) / sampling_rate
    
    # Convert to beats per minute
    heart_rates = 60 / intervals
    
    # Return average heart rate
    return np.mean(heart_rates)

# Generate a synthetic ECG-like signal for demonstration
def generate_synthetic_ecg(duration_sec=10, sampling_rate=250):
    """Generate a synthetic ECG-like signal"""
    t = np.arange(0, duration_sec, 1/sampling_rate)
    heart_rate = 60  # 60 BPM
    beat_interval = 60 / heart_rate
    
    # Create basic ECG shape
    ecg = np.zeros_like(t)
    for beat_time in np.arange(0, duration_sec, beat_interval):
        # Find indices close to each heartbeat
        idx = np.where((t >= beat_time) & (t <= beat_time + 0.2))[0]
        if len(idx) > 0:
            # Create an R peak
            peak_idx = idx[0]
            if peak_idx < len(ecg):
                ecg[peak_idx] = 1.0
                # Add Q and S waves
                if peak_idx > 0:
                    ecg[peak_idx-1] = -0.2
                if peak_idx < len(ecg) - 1:
                    ecg[peak_idx+1] = -0.2
                # Add T wave
                t_idx = np.where((t >= beat_time + 0.1) & (t <= beat_time + 0.3))[0]
                if len(t_idx) > 0:
                    ecg[t_idx] += 0.3 * np.exp(-(t[t_idx] - (beat_time + 0.2))**2 / 0.01)
    
    # Add some noise
    ecg += np.random.normal(0, 0.05, len(t))
    
    return t, ecg

# Generate and process synthetic ECG
t, ecg_signal = generate_synthetic_ecg()
sampling_rate = 250  # Hz

# Apply moving average to filter noise
filtered_ecg = moving_average(ecg_signal, 5)

# Find R peaks
threshold = 0.5
r_peaks = find_peaks(filtered_ecg, threshold)

# Calculate heart rate
hr = calculate_heart_rate(r_peaks, sampling_rate)

# Plot the results
plt.figure(figsize=(12, 6))
plt.subplot(211)
plt.plot(t, ecg_signal, 'b-', label='Raw ECG')
plt.plot(t[:len(filtered_ecg)], filtered_ecg, 'r-', label='Filtered ECG')
plt.plot(t[r_peaks], filtered_ecg[r_peaks], 'go', label='Detected R-peaks')
plt.legend()
plt.title('ECG Signal Processing')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.grid(True)

plt.subplot(212)
plt.stem(t[r_peaks], filtered_ecg[r_peaks])
plt.title(f'R-peaks (Heart Rate: {hr:.1f} BPM)')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.grid(True)

plt.tight_layout()
plt.show()

print(f"Calculated heart rate: {hr:.1f} BPM")

## 2. Classes for Biomedical Data Representation

Object-oriented programming is particularly useful in biomedical applications where data has natural hierarchical structures. For example, a patient has medical records, each containing multiple measurements or signals.

In [ ]:
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt

class Patient:
    """Class representing a patient with medical records"""
    
    def __init__(self, patient_id, name, birth_date, gender):
        self.patient_id = patient_id
        self.name = name
        self.birth_date = birth_date
        self.gender = gender
        self.medical_records = []
    
    def add_record(self, record):
        """Add a medical record to this patient"""
        if not isinstance(record, MedicalRecord):
            raise TypeError("Must be a MedicalRecord instance")
        self.medical_records.append(record)
    
    def get_age(self):
        """Calculate patient's age in years"""
        today = datetime.now()
        born = datetime.strptime(self.birth_date, "%Y-%m-%d")
        return today.year - born.year - ((today.month, today.day) < (born.month, born.day))
    
    def __str__(self):
        return f"Patient {self.patient_id}: {self.name}, {self.get_age()} years, {self.gender}"


class MedicalRecord:
    """Class representing a medical record with measurements"""
    
    def __init__(self, record_id, date, doctor):
        self.record_id = record_id
        self.date = date
        self.doctor = doctor
        self.measurements = {}
        self.signals = {}
    
    def add_measurement(self, name, value, unit):
        """Add a single measurement with unit"""
        self.measurements[name] = {"value": value, "unit": unit}
    
    def add_signal(self, name, signal, sampling_rate, unit):
        """Add a physiological signal"""
        self.signals[name] = {
            "data": signal, 
            "sampling_rate": sampling_rate, 
            "unit": unit
        }
    
    def get_measurement(self, name):
        """Retrieve a measurement"""
        if name in self.measurements:
            return self.measurements[name]
        return None
    
    def plot_signal(self, name):
        """Plot a signal if available"""
        if name not in self.signals:
            print(f"Signal '{name}' not found")
            return
            
        signal = self.signals[name]
        data = signal["data"]
        time = np.arange(len(data)) / signal["sampling_rate"]
        
        plt.figure(figsize=(10, 4))
        plt.plot(time, data)
        plt.title(f"{name} Signal")
        plt.xlabel("Time (s)")
        plt.ylabel(f"Amplitude ({signal['unit']})")
        plt.grid(True)
        plt.show()
    
    def __str__(self):
        return f"Medical Record {self.record_id} from {self.date} by Dr. {self.doctor}"


class BiologicalSignal:
    """Class for processing and analyzing biological signals"""
    
    def __init__(self, data, sampling_rate, signal_type="Unknown"):
        self.data = np.array(data)
        self.sampling_rate = sampling_rate
        self.signal_type = signal_type
    
    def filter(self, window_size=5):
        """Apply moving average filter"""
        return np.convolve(self.data, np.ones(window_size)/window_size, mode='valid')
    
    def find_peaks(self, threshold, min_distance=1):
        """Find peaks in the signal"""
        peaks = []
        for i in range(1, len(self.data)-1):
            if (self.data[i] > threshold and 
                self.data[i] > self.data[i-1] and 
                self.data[i] > self.data[i+1]):
                if not peaks or i - peaks[-1] >= min_distance:
                    peaks.append(i)
        return peaks
    
    def get_time_array(self):
        """Get time array for the signal"""
        return np.arange(len(self.data)) / self.sampling_rate
    
    def plot(self, title=None):
        """Plot the signal"""
        plt.figure(figsize=(10, 4))
        plt.plot(self.get_time_array(), self.data)
        plt.title(title or f"{self.signal_type} Signal")
        plt.xlabel("Time (s)")
        plt.ylabel("Amplitude")
        plt.grid(True)
        plt.show()


# Example usage
if __name__ == "__main__":
    # Create a patient
    patient = Patient("P12345", "John Doe", "1980-05-15", "M")
    
    # Create a medical record
    record = MedicalRecord("R001", "2023-01-20", "Smith")
    
    # Add measurements
    record.add_measurement("heart_rate", 72, "bpm")
    record.add_measurement("blood_pressure_systolic", 120, "mmHg")
    record.add_measurement("blood_pressure_diastolic", 80, "mmHg")
    record.add_measurement("temperature", 36.8, "°C")
    
    # Generate synthetic ECG and add as signal
    _, ecg_data = generate_synthetic_ecg(duration_sec=5, sampling_rate=250)
    record.add_signal("ECG", ecg_data, 250, "mV")
    
    # Add the record to the patient
    patient.add_record(record)
    
    # Display patient info
    print(patient)
    print(f"Number of medical records: {len(patient.medical_records)}")
    
    # Display record info
    print(record)
    print("Measurements:")
    for name, data in record.measurements.items():
        print(f"  - {name}: {data['value']} {data['unit']}")
    
    # Plot the ECG signal
    record.plot_signal("ECG")
    
    # Create a BiologicalSignal object for more advanced processing
    ecg_signal = BiologicalSignal(ecg_data, 250, "ECG")
    filtered_ecg = ecg_signal.filter(window_size=5)
    peaks = ecg_signal.find_peaks(threshold=0.5, min_distance=50)
    
    # Plot with peaks
    plt.figure(figsize=(10, 4))
    plt.plot(ecg_signal.get_time_array(), ecg_signal.data, 'b-', label='ECG')
    plt.plot(np.array(peaks)/250, ecg_signal.data[peaks], 'ro', label='R-peaks')
    plt.title("ECG with Detected R-peaks")
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude (mV)")
    plt.legend()
    plt.grid(True)
    plt.show()

## 3. Virtual Environments for Biomedical Software Development

Virtual environments are essential for biomedical software development as they allow us to:

1. Create isolated environments for different projects
2. Manage dependencies without conflicts
3. Ensure reproducibility of analysis results
4. Share environment specifications with collaborators

Below we'll cover how to set up and manage Python virtual environments for biomedical applications.